In [48]:
import textwrap
from typing import Any, Callable, Generic, TypeVar, cast
from typing_extensions import Self

_T = TypeVar("_T")

class Module(Generic[_T]):
    owner: _T

    @classmethod
    def create_for_owner(cls, owner: _T, *args, **kwargs) -> Self:
        obj = cast(Self, cast(Callable, cls)(*args, **kwargs))
        obj.owner = owner
        return obj

    def __repr__(self) -> str:
        return f"<{type(self).__name__} of {self.owner}>"

    def __str__(self) -> str:
        return f"{type(self).__name__} of {self.owner}"

    def __eq__(self, other: Any) -> bool:
        return (
            isinstance(other, type(self))
            and self.owner == other.owner
        )

    def __hash__(self) -> int:
        return hash((type(self), self.owner))


_M = TypeVar("_M", bound=Module)

class ModuleEntry(Generic[_M]):
    def __init__(self, module_class: type[_M], *args, **kwargs) -> None:
        self.module_class = module_class
        self.args = args
        self.kwargs = kwargs

    def create_for_owner(self, owner: Any):
        return self.module_class.create_for_owner(owner, *self.args, **self.kwargs)

class Modular:
    __module_entries: list[ModuleEntry[Module[Self]]] = []
    
    __modules: list[Module[Self]]

    def __init__(self) -> None:
        self.__modules = []
        for module_entry in self.__module_entries:
            self.__modules.append(module_entry.create_for_owner(self))

    @classmethod
    def add_module_class(cls, module_class: type[Module[Self]], *args, **kwargs):
        if cls.has_module_class(module_class):
            raise ValueError(f"{module_class.__name__} already added.")

        cls.__module_entries.append(ModuleEntry(module_class, *args, **kwargs))

    @classmethod
    @property
    def module_entries(cls):
        return tuple(cls.__module_entries)

    @classmethod
    def has_module_class(cls, module_class: type):
        for entry in cls.__module_entries:
            if entry.module_class == module_class:
                return True
        return False

    def module(self, module_class: type[_M]) -> _M:
        for module in self.__modules:
            if isinstance(module, module_class):
                return module

        if type(self).has_module_class(module_class):
            raise ValueError(f"Module class {module_class.__name__} was added after object was instantiated.")

        raise ValueError(f"{type(self).__name__} does not have module {module_class}.")

    def __rmatmul__(self, other: type[_M]) -> _M:
        return self.module(other)

In [49]:
class LabelModule(Module):
    """A module which provides a label."""

    label: str

    def __init__(self, label: str) -> None:
        self.label = label

    def __str__(self) -> str:
        return self.label

In [50]:
class Foo(Modular):
    def __str__(self) -> str:
        return "Foo"

Foo.add_module_class(LabelModule)

In [51]:
foo = Foo()
print(foo.module(LabelModule).label)

TypeError: __init__() missing 1 required positional argument: 'label'